In [ ]:
import os.path
%run 0-0-common.ipynb

In [ ]:
%%time

subs = ["SFWRedheads","sfwpetite","SFWNextDoorGirls", "celebrities", "wallstreetbets", "fatsquirrelhate","CityPorn","EarthPorn","memes","spaceporn","trippinthroughtime","fatsquirrelhate","oldladiesbakingpies","itookapicture", "celebrities", "realasians", "KoreanHotties", "prettyasiangirls","AsianOfficeLady", "mildlypenis"]

extant_data: pd.DataFrame = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

distinct_data: pd.DataFrame = extant_data.drop_duplicates(subset=['id', 'title']).reset_index(drop=True)

target_frame: pd.DataFrame = distinct_data[distinct_data['subreddit'].isin(subs)]

sorted_target_frame = target_frame.sort_values(by=['title'], ascending=False)

display(sorted_target_frame)

In [ ]:
%%time

target_frame['exists'] = target_frame.apply(lambda row: check_if_exists(row['thumbnail_path']), axis=1)

exists = target_frame.where(target_frame['exists']).dropna(how='all').reset_index(drop=True)

display(exists)

In [ ]:
exists.where(lambda x: x['caption'] != "").dropna(how='all').reset_index(drop=True)
display(exists)

In [ ]:
display(sources_df)

In [ ]:
smaller_exportable_df = pd.DataFrame(
	data=[exists['path'], exists['image_name'], exists['caption'], exists['title'], exists['subreddit']],
	index=['path', 'image_name', 'caption', 'title', 'subreddit']).T

display(smaller_exportable_df)

In [ ]:
smaller_exportable_df['name'] = smaller_exportable_df.progress_apply(lambda x: add_source(x, sources), axis=1)
display(smaller_exportable_df)

In [ ]:
!mkdir CelebrityDiffusion

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar():
	dask_frame = dd.from_pandas(smaller_exportable_df, npartitions=10)
	smaller_exportable_df['image_data'] = dask_frame.apply(lambda x: move_file(x), meta=('str', object),axis=1).compute()
display(smaller_exportable_df)

In [ ]:
smaller_exportable_df.to_parquet("celebrities.parquet")

In [ ]:
extant_files = os.listdir("CelebrityDiffusion")

things = []
records = smaller_exportable_df.to_dict('records')
for record in records:
	if record['image_name'] in extant_files:
		image_name = record['image_name']
		text = record['title']
		caption = record['caption']
		out_record = {"file_name": image_name, "text": [text, caption]}
		things.append(out_record)

pd.DataFrame(things).to_json(f"CelebrityDiffusion/metadata.jsonl", orient="records", lines=True)

In [ ]:
!type CelebrityDiffusion\metadata.jsonl

In [ ]:
!powershell Compress-Archive CelebrityDiffusion CelebrityDiffusion.zip

In [ ]:
len(extant_files)